In [2]:
import re
import time
from bs4 import BeautifulSoup
import requests
import json
#import Levenshtein as lv
import numpy as np
from gensim import corpora, models, similarities
import spacy


#for findDOI
import feedparser
import jellyfish
import copy

In [6]:
from rss import rss # список рассылки
from findDOI import findDOI # функция поиска по doi по названию
from ALT import altmetrics # функция поиска альтметрик 

In [5]:
altmetrics('10.1021/jacs.9b02936' ) 

200


{'title': 'A General Strategy for Fabricating Isolated Single Metal Atomic Site Catalysts in Y Zeolite',
 'doi': '10.1021/jacs.9b02936',
 'pmid': '31117651',
 'isbns': [],
 'altmetric_jid': '4f6fa4f63cf058f610002f56',
 'issns': ['0002-7863', '1520-5126'],
 'journal': 'Journal of the American Chemical Society',
 'cohorts': {'sci': 2},
 'context': {'all': {'count': 13029364,
   'mean': 7.9319531459684,
   'rank': 10391646,
   'pct': 11,
   'higher_than': 1452108},
  'journal': {'count': 47084,
   'mean': 5.378367053926,
   'rank': 43335,
   'pct': 4,
   'higher_than': 2305},
  'similar_age_3m': {'count': 133785,
   'mean': 11.890596513784,
   'rank': 94920,
   'pct': 17,
   'higher_than': 23820},
  'similar_age_journal_3m': {'count': 237,
   'mean': 7.5265762711864,
   'rank': 202,
   'pct': 6,
   'higher_than': 15}},
 'authors': ['Yiwei Liu',
  'Zhi Li',
  'Qiuying Yu',
  'Yanfei Chen',
  'Ziwei Chai',
  'Guofeng Zhao',
  'Shoujie Liu',
  'Weng-Chon Cheong',
  'Yuan Pan',
  'Qinghua Zha